## استيراد المكتبات المطلوبة  
استيراد مكتبات PyTorch و transformers اللازمة لتحميل واستخدام نموذج Phi-4.  


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## ضبط البذرة العشوائية  
قم بضبط البذرة العشوائية لضمان الحصول على نتائج قابلة للتكرار عبر تشغيلات مختلفة.


In [ ]:
torch.random.manual_seed(0)

## تحميل نموذج Phi-4-mini-flash-reasoning والمحلل اللغوي
قم بتحميل نموذج Microsoft Phi-4-mini-flash-reasoning والمحلل اللغوي الخاص به من Hugging Face. سيتم تحميل النموذج على CUDA للحصول على استنتاج أسرع.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## تحضير رسالة الإدخال  
قم بإنشاء رسالة محادثة تحتوي على مسألة رياضية تتعلق بمعادلة تربيعية، وقم بتنسيقها باستخدام قالب المحادثة الخاص بالنموذج.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## إنشاء استجابة  
إنشاء استجابة من النموذج باستخدام معايير محددة مثل درجة الحرارة و top_p للتحكم في العشوائية في الناتج.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## فك تشفير المخرجات إلى نص
قم بتحويل تسلسل الرموز الناتجة إلى نص يمكن قراءته بواسطة الإنسان، مع استثناء الرموز الأصلية المدخلة لعرض استجابة النموذج فقط.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالترجمة البشرية الاحترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
